# Fairness in AI: Removing word embeddings

#### Kylian van Geijtenbeek, Thom Visser, Martine Toering, Iulia Ionescu

$\textbf{Abstract:}$ In this paper we reproduce the word embedding debiasing algo-rithm from Bolukbasi et al. [2]. We adapt their online codebase and extend it with their soft debiasing method, integrate several popular benchmarks and investigate the effectiveness of the algorithmon the newer fastText, BERT and XLNet embeddings, besides the Word2vec embeddings used by Bolukbasi et al. [2]. We show that the removal of direct bias from all the different embeddings barely affects their effectiveness through a comparison of benchmark scores. However, we fail to reproduce the large scale soft debiasing results due to a lack of detail on the original implementation.

In [1]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np
import copy

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import *

from debiaswe.debias import *
from debiaswe.benchmarks import Benchmark

## Notebook preferences

# 1 - Gender Bias in word2vec, Glove and FastText

### Load data

In this notebook, we will use three different word embeddings: $\textbf{word2vec}$ (Mikolov et al. 2013), $\textbf{glove}$ (Pennington et al. 2014) and $\textbf{fastText}$ (Bojanowski et al. 2016).

The word2vec embedding we use is learned from a corpus of Google News articles (https://code.google.com/archive/p/word2vec/). The embeddings are 300-dimensional for 3 million words. For glove we make use of the 300-dimensional vectors trained on Common Crawl (https://nlp.stanford.edu/projects/glove/). Lastly, FastText is a word embedding from Facebook AI Research lab trained on Wikipedia corpus and Common Crawl and also consists of 300-dimensional vectors.

We start by loading in the data.

In [3]:
# TODO 

# Load google news word2vec
E = WordEmbedding("./embeddings/w2v_gnews_small.txt")
# Load Glove
# E_g = WordEmbedding(glove)
# Load FastText 
# E = WordEmbedding("fasttext_small")

Creating embedding from file: /home/martine/FactAI/replication/embeddings/w2v_gnews_small.txt
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
Embedding shape: (26423, 300)


In [ ]:
# Load professions and gender related lists from Bolukbasi et al. 
profession_words, defs, equalize_pairs = load_data()

### Define gender direction

In [ ]:
# Define gender direction with the words "she" and "he" 
v_gender = E.diff('she', 'he')

# Define gender direction with PCA
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
    
# Only keep definitional pairs that are present in the word embedding
defs = [d for d in defs if d[0] in E.words and d[1] in E.words]

# v_gender = we.doPCA(defs, E).components_[0]

### Generating analogies


In [ ]:
# Analogies gender
a_gender = E.best_analogies_dist_thresh(v_gender, thresh=1)

# Visualising
we.viz(a_gender)

### Analyzing occupational gender bias 


In [ ]:
# Analysis of extreme male and extreme female professions
sp = E.profession_stereotypes(profession_words, v_gender)

# 2 - Comparing Bias of word2vec, Glove and FastText

We will compare the gender bias between word embeddings FastText and Glove. We do this by following Bolukbasi et al. approach. The profession words are projected onto the gender axis for FastText and Glove. Each datapoint represents a profession word.

In [ ]:
# To compare word2vec to a second word embedding, change this to either fastText (E_f) or Glove (E_g)
compare_embedding = "fastText"
E_compare = E_f

In [ ]:
def get_datapoints(sp, E, v_gender, words_set):
    sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words if w in words_set])
    points = [s[0] for s in sp]
    words = [s[1] for s in sp]
    words_sorted_ind = sorted(range(len(words)), key=lambda k: words[k])
    datapoints =  [points[i] for i in words_sorted_ind]
    return datapoints

In [ ]:
# Define gender direction and analyse occupational bias in other word embedding
v_gender_compare = E_compare.diff('she', 'he')
sp_compare = E_compare.profession_stereotypes(profession_words, v_gender_compare)

words_set = [s[1] for s in sp_compare]
compare_datapoints = get_datapoints(sp_compare, E_compare, v_gender_compare, words_set)
og_datapoints = get_datapoints(sp, E, v_gender, words_set)

In [ ]:
# Plot comparison of bias of word2vec and fastText (or Glove)
fig, ax = plt.subplots()
ax.scatter(compare_datapoints, og_datapoints, s=10)
ax.set_ylim(-0.3, 0.5)
ax.set_xlim(-0.3, 0.5)
plt.xlabel("Gender axis of {}".format(compare_embedding), fontsize=11)
plt.ylabel("Gender axis of word2vec", fontsize=11)
plt.title("Gender bias in profession words of embeddings", pad=18, fontsize=13)
fig.savefig("comparing_bias.png")
plt.show()

# 3 - Debiasing algorithms on word2vec, Glove and FastText

## Hard debiasing

In hard debiasing, the gender neutral words are shifted to zero in the gender subspace (i.e. neutralized) by subtracting the projection of the neutral word embedding vector onto the gender subspace and renormalizing the resulting embedding to unit length. 

## Soft debiasing

We adapted specifics from Manzini et al., Soft debiasing is done by solving the following optimization problem as mentioned in their paper:

\begin{equation}
    \underset{T}{\min} || (TW)^T(TW) - W^TW||^2_F + \lambda ||(TN)^T (TB)||^2_F
\end{equation}

where W is the matrix of all embedding vectors, N is the matrix of the embedding vectors of the gender neutral words, B is the gender subspace, and T is the debiasing transformation that minimizes the projection of the neutral words onto the gender subspace but tries to maintain the pairwise inner products between the words.

This code is largely based on code from https://github.com/TManzini/DebiasMulticlassWordEmbedding.

Please choose one option for debiasing in the following cells.

In [ ]:
# TODO 

# Change this to dataset to debias for fastText (E_f) or Glove (E_g). Default is word2vec (E)
E_debias = E

In [ ]:
# Hard debias
hard_debias(E, gender_specific_words, defs, equalize_pairs)

# Soft debias
# soft_debias(E, gender_specific_words, defs, log=False)

In [ ]:
# TODO

# Save hard debiased embeddings
E.save('./embeddings/w2v_gnews_small_hard_debiased.txt')

# Save soft debiased embeddings
# E.save('./embeddings/w2v_gnews_small_soft_debiased.txt')

In [ ]:
# Analysis of extreme male and extreme female professions
sp_debiased = E.profession_stereotypes(profession_words, v_gender)

In [ ]:
# Analogies gender
# a_gender_debiased = E.best_analogies_dist_thresh(v_gender)

# Visualising
# we.viz(a_gender_debiased)

# 4 - Benchmarks

In [ ]:
# TODO

def run_benchmark(E, embedding_name):
    benchmark = Benchmark()
    result_original = benchmark.evaluate(E, "'Before', {}".format(embedding_name))
    E_hard = copy.deepcopy(E)
    hard_debias(E_hard, gender_specific_words, defs, equalize_pairs)
    result_hard_debiased = benchmark.evaluate(E_hard, "'Hard debiased', {}".format(embedding_name))
    E_soft = copy.deepcopy(E) 
    soft_debias(E_soft, gender_specific_words, defs, log=False)
    result_soft_debiased = benchmark.evaluate(E, "'Soft debiased', {}".format(embedding_name))
    results = [result_original, result_hard_debiased, result_soft_debiased]
    return results

In [ ]:
# Evaluate for word2vec, Glove and FastText
w2v_results = run_benchmark(E, "word2vec")
# g_results = run_benchmark(E_g, "Glove")
# f_results = run_benchmark(E_f, "fastText")

benchmark.pprint_compare(w2v_results, ["Before", "Hard-debiased", "Soft-debiased"], "word2vec")
# benchmark.pprint_compare(g_results, ["Before", "Hard-debiased", "Soft-debiased"], "Glove")
# benchmark.pprint_compare(f_results, ["Before", "Hard-debiased", "Soft-debiased"], "fastText")